In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('../data/data.csv')
data.head()

In [ ]:
def analysis(data: pd.DataFrame):
    pass

In [ ]:
def feature_engineering(data: pd.DataFrame) -> pd.DataFrame:
    pass

In [ ]:
def create_model(data: pd.DataFrame):
    pass

In [ ]:
def cv(data: pd.DataFrame):
    pass